In [ ]:
import os
import pandas as pd
import cv2
import numpy as np 
import pandas as pd 
import time
import matplotlib.pyplot as plt
from skimage import measure
import matplotlib.patches as patches
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# from sklearn.utils import resample
# from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils import resample
from keras.preprocessing.image import ImageDataGenerator
#imports
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten,Conv2D,GlobalMaxPooling2D
from keras import optimizers
from keras.preprocessing import image
from keras.metrics import top_k_categorical_accuracy
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet50 import ResNet50,preprocess_input
from keras.applications.xception import Xception
from keras.optimizers import Adam
from keras import regularizers

print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

In [ ]:
DATA="../input/humpback-whale-identification"
TRAIN_IMG="../input/humpback-whale-identification/train"
TEST_IMG="../input/humpback-whale-identification/test"

In [ ]:
test_df= pd.DataFrame({"Image":  os.listdir(TEST_IMG)})
print("test images:"+ str(len(test_df)))

In [ ]:
train_lbl = pd.read_csv(os.path.join(DATA, 'train.csv'))
print("train images:"+ str(len(train_lbl)))
print("total unique class:"+ str(len(np.unique(train_lbl['Id']))))
train_lbl.head()

**Balance training set**

In [ ]:
train_lbl['newwhale_lbl']=np.where(train_lbl['Id']=='new_whale',1,0)

In [ ]:
#take out whales with less than 2 examples
df=train_lbl.groupby(['Id']).size().reset_index(
    name='train_examples')
df=df[df['train_examples']>=2]

print("number of classes with more than 2 examples:"+ str(len(df)))
train_lbl=train_lbl[train_lbl['Id'].isin(df['Id'])]
print("number of train instances :"+ str(len(train_lbl)))

In [ ]:
#define accuracy metric
def top_5_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

Train model without new whale

In [ ]:
train_no_new_whale=train_lbl[train_lbl['newwhale_lbl']!=1]
print("number of train instances :"+ str(len(train_no_new_whale)))

In [ ]:
image_size=100
batch_size=32

datagen=image.ImageDataGenerator(rescale=1./255,
                                 validation_split = 0.2,
#                                  preprocessing_function=preprocess_input,
#                                  check augmentation options on training:
                                 rotation_range=20,
#                                  zoom_range=2,
                                 shear_range=0.2,
                                 horizontal_flip=True,
#                                  rescale=1./255, 
                                 fill_mode='nearest')

#training set
train_generator=datagen.flow_from_dataframe(dataframe=train_no_new_whale, 
                                            directory=TRAIN_IMG, 
                                            x_col="Image", y_col="Id", 
                                            has_ext=True, seed = 42,
                                            class_mode="categorical", 
                                            shuffle=True,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            subset = "training")

validation_generator = datagen.flow_from_dataframe(dataframe=train_no_new_whale,
                                                   directory=TRAIN_IMG, 
                                                   x_col="Image", y_col="Id", 
                                                   has_ext=True, seed = 42,shuffle=True,
                                                   class_mode="categorical", 
                                                   target_size=(image_size,image_size), 
                                                   batch_size=1, subset = "validation")

test_datagen = image.ImageDataGenerator(rescale=1./255
#                                         preprocessing_function=preprocess_input
                                       )
test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                                  directory=TEST_IMG, 
                                                  x_col="Image",y_col=None,
                                                  seed = 42, class_mode=None, 
                                                  target_size=(image_size,image_size), 
                                                  batch_size=1,shuffle = False)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.utils.vis_utils import plot_model
from PIL import Image as pil_image

def cnn():
    CLASSES = len(np.unique(train_no_new_whale['Id'])) 
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides = (1, 1), input_shape = (image_size, image_size, 3)))
    model.add(BatchNormalization(axis = 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), strides = (1,1)))
    model.add(Activation('relu'))
    model.add(AveragePooling2D((3, 3)))
    model.add(Flatten())
    model.add(Dense(500, activation="relu",kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.4))
    model.add(Dense(CLASSES, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy',top_5_accuracy])
    # visualize
    model.summary()
    
    return model

model = cnn()


In [ ]:
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True,expand_nested=True)
    pil_image.open('model_plot.png')

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
epochs=100

checkpointer = ModelCheckpoint(filepath='weights_cnn_no_new_whale.hdf5', monitor='loss',
                               verbose=1, save_best_only=True)
# decayedlr= ReduceLROnPlateau(monitor='val_loss',patience=10,min_lr=1e-9,verbose=1,mode='min')

model.save('cnn_no_new_whale.h5')

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=epochs, callbacks = [checkpointer])

model.save('cnn_no_new_whale.h5')

In [ ]:
# model.evaluate_generator(generator=validation_generator)

In [ ]:
plt.plot(history.history['top_5_accuracy'])
plt.plot(history.history['val_top_5_accuracy'])
plt.title('model top 5 accuracy')
plt.ylabel('top 5 accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# # from keras.models import model_from_json, load_model
# # model=load_model("../input/pretrained-model/cnn_no_new_whale.h5",custom_objects={"top_5_accuracy": top_5_accuracy})
# history = model.fit_generator(generator=train_generator,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=validation_generator,
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=epochs, callbacks = [checkpointer])

In [ ]:
# model.save('cnn_no_new_whale.h5')

# Submission

In [ ]:
#submission
STEP_SIZE_TEST=test_generator.n
test_generator.reset()
pred = model.predict_generator(test_generator,verbose = 1,steps=STEP_SIZE_TEST,workers=1)
pred_sorted = np.argsort(-pred, axis = 1)[:,:5]
pred_sorted_big = np.argsort(-pred, axis = 1)[:,:100]
top_pred=np.sort(-pred, axis = 1)[:,:1]
top_5_pred=np.sort(-pred, axis = 1)[:,:5]
# pred_sorted

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
test_filenames=test_generator.filenames

In [ ]:
from tqdm import tqdm
#create empty list
pred_ids = list()
for i,row in enumerate(tqdm(pred_sorted)):
    #create a temporary list to store the ids for a given image
    temp_list = []
    for j,predlabel in enumerate(row):
        #for each index in pred_sorted, append the real Id in temp_list
        temp_list.append(labels[predlabel])
    pred_ids.append(temp_list)

In [ ]:
### for pred big file
pred_ids_big = list()
for i,row in enumerate(tqdm(pred_sorted_big)):
    #create a temporary list to store the ids for a given image
    temp_list = []
    for j,value in enumerate(row):
        #for each index in pred_sorted, append the real Id in temp_list
        temp_list.append(labels[value])
    #append all 5 ids for a given image to pred_ids
    #effectively creating a similar list to pred_sorted
    #but with the real ids
    pred_ids_big.append(temp_list)

In [ ]:
final_preds = []
for i,top_5_ids in enumerate(pred_ids):
    final_preds.append(' '.join(pred_ids[i]))
    
import shutil
shutil.rmtree('test_folder', ignore_errors=True)

submission = pd.DataFrame({"Image": test_filenames, "Id": final_preds})
submission.to_csv("submission.csv", index = False) #disabled to not override current submission
submission.head()

In [ ]:
### for pred big file
final_preds = []

for i,top_5_ids in enumerate(pred_ids_big):
    final_preds.append(' '.join(pred_ids_big[i]))
    
import shutil
shutil.rmtree('test_folder', ignore_errors=True)

top_pred=-1*top_pred
top_5_pred=-1*top_5_pred

submission_top100 = pd.DataFrame({"Image": test_filenames,"top_score":list(top_pred), "Id": final_preds})
submission_top100.to_csv("submission_top100.csv", index = False)
submission_top100.head(50)

submission_scores=  pd.DataFrame({"Image": test_filenames,"1_score":list(top_5_pred[:,0]),"2_score":list(top_5_pred[:,1]),"3_score":list(top_5_pred[:,2]),"4_score":list(top_5_pred[:,3]),"5_score":list(top_5_pred[:,4])})
submission_scores.to_csv("submission_scores_top5.csv", index = False)

In [ ]:
submission_top100.head(50)

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index = False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe


# create a link to download the dataframe
create_download_link(submission)


In [ ]:
create_download_link(submission_scores)